In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import Runnable
from langchain.schema.runnable.config import RunnableConfig
import chainlit as cl
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

#######
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./testdir/2304.08485.pdf")
pages = loader.load_and_split()

from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=pages, embedding=GPT4AllEmbeddings())

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#######

2024-04-08 10:23:52 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [32]:
question = "What is the title of the paper?"
docs = vectorstore.similarity_search(question)  # Perform the search based on the query
format_docss = format_docs(docs)

for doc in docs: 
    new_doc = doc.page_content



AttributeError: 'str' object has no attribute 'page_content'

In [2]:
@cl.on_chat_start
async def on_chat_start():
    model = ChatOllama(model="llama2")
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You're a troubleshooting guide for semiconductor manufacturing tools.",
            ),
            ("human", "{question}"),
        ]
    )
    runnable = {"docs": format_docs} | prompt | model | StrOutputParser()
    cl.user_session.set("runnable", runnable)


In [3]:
@cl.on_message
async def on_message(message: cl.Message):
    question = message.content  # The user's query is now defined as 'question'
    docs = vectorstore.similarity_search(question)  # Perform the search based on the query
    formatted_docs = format_docs(docs)  # Format the retrieved documents
    
    # Now, integrate `formatted_docs` with your chat model as context for generating a response.
    # This part might require adjustments based on the expected format of `ChatOllama` and runnable configuration.

    runnable = cl.user_session.get("runnable")  # Ensure this is correctly initialized and configured to use `formatted_docs`.

    msg = cl.Message(content="")

    async for chunk in runnable.astream(
        {"question": message.content, "context": formatted_docs},  # Assuming `astream` can accept additional context.
        config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
    ):
        await msg.stream_token(chunk)

    await msg.send()